### Запуск QtLab

Для начала надо добавить пути со скриптами и драйверами устройств, подключить библиотеки numpy и matplotlib.

In [1]:
%matplotlib qt5
from qsweepy import *
from qsweepy.instruments import *
from qsweepy.save_pkl import save_pkl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
interactive(True)
import imp
import qsweepy.save_pkl

Failed loading module Keysight_M3102A:  No module named 'keysightSD1'
Failed loading module Keysight_M3202A:  No module named 'keysightSD1'

Failed loading module Signal_Hound_SA:  name 'signal_hound_dll' is not defined


## Параметры кубитов

In [70]:
#Параметры кубитов.all keys should be str
Qubits = {'1':{'r':{'Fr': 7.278e9, 'P': -3},
               'q':{'00-1-01': 4.246158e9, 
                    '00-2-02': 4.12454e9,
                    '00-2-11': 4.09916e9,
                    '01-1-02': 4.0030e9, 
                    '00-1-10': 3.94780e9,
                    '00-2-20': 3.82293e9,
                    '10-1-20': 3.6990e9}},
        '2':{'r':{'Fr': 7.332e9, 'P':-10}, 
               'q':{'00-1-01': 4.250412e9,#4.24947e9,
                    '00-2-02': 4.12927e9,#4.129e9,  
                    '00-2-11': 4.09495e9,
                    '01-1-02': 4.0147e9, 
                    '00-1-10': 3.93139e9,#3.9316e9,
                    '00-2-20': 3.80671e9,#3.80683e9,
                    '10-1-20': 3.6762e9}}}
qubit_params=Qubits
qubit_id = '2'
#it should be dumped too
#qubits_dump = {"qubits":qubit_params,'switch':switch_to_sample}
#qjson.dump('qubits','qubit_params',qubits_dump)

## измерительные устройства

In [5]:
# RF switch for making sure we know what sample we are measuring
pna = Agilent_N5242A('pna', address = 'TCPIP0::10.20.61.48::inst0::INSTR')
lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')
rf_switch = nn_rf_switch('rf_switch', address='10.20.61.224')
awg = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
adc = Spectrum_M3i2132('adc')
sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
lo_ex = lo1
lo_ro = pna
awg_tek=awg

## Спектроскопия

In [64]:
# векторный анализатор в режим линейного свипа
pna.set_sweep_mode("LIN")
# Смесители -- в режим пропускания
for channel in range(1,5): #AWG5014C
#for channel in range(0,4): # M3202A
    awg.set_amplitude(.2, channel=channel)
    awg.set_offset(0.1, channel=channel)
    awg.set_output(1, channel=channel)
    awg.set_waveform([0], channel=channel)

# Однотоновая спектроскопия резонатора в зависимости от мощности


In [68]:
lo1.set_status(0)
pna.set_status(1)
#pna.set_power(-35) #
powers = np.linspace(10, -20, 31)
pna.set_nop(101)
pna.set_xlim(7.25e9, 7.35e9)
pna.set_bandwidth(30)
#freqs = pna.get_freqpoints()
#S21 = pna.get_tracedata()
meas = sweep.sweep(pna, (powers, pna.set_power, 'Мощность, дБ'), filename='Однотоновая спектроскопия, образец #{}'.format(qubit_id))

A Jupyter Widget

Started at:  Aug 29 2018 16:42:44
Exiting sweep main thread #{} via return 2932

Elapsed time: 0 h 1 m 38.04 s


## построение графиков задержки

In [69]:
delay_median = np.median(np.gradient(np.angle(meas['S-parameter'][2]), axis=1))
phase_compensator = np.exp(-1j*delay_median*np.arange(meas['S-parameter'][2].shape[1]))
meas_delay_compensated = meas['S-parameter'][2]*phase_compensator
meas_phase_median = np.median(np.angle(meas['S-parameter'][2]), axis=1)
#plt.figure('Однотоновая спектроскопия резонатора в зависимости от мощности')
meas_compensated = {'S-parameter':[i for i in meas['S-parameter']]}
meas_compensated['S-parameter'][2] = meas['S-parameter'][2]*phase_compensator

frequencies = meas_compensated['S-parameter'][1][1]/1e9
ang_freq_step = 2*np.pi*(frequencies[1]-frequencies[0])

#plotting.plot_measurement(meas_compensated)
plt.figure(figsize=(12,6))
plt.title('Однотоновая спектроскопия резонатора в зависимости от мощности, образец #{}'.format(qubit_id))
plt.pcolormesh(meas_compensated['S-parameter'][1][0], 
               meas_compensated['S-parameter'][1][1]/1e9, 
               -np.gradient(np.unwrap(np.angle(meas_compensated['S-parameter'][2])).T)[0]/ang_freq_step, cmap='RdBu')
plt.colorbar()
#plt.close()
plt.xlabel('Мощность, дБ')
plt.ylabel('Частота, ГГц')
plt.tight_layout()
plt.grid(True)
plt.savefig('single-tone-spectroscopy.pdf')


plt.figure(figsize=(12,6))
plt.title('Однотоновая спектроскопия резонатора (-10 дБм), образец #{}'.format(qubit_id))
plt.plot(meas_compensated['S-parameter'][1][1]/1e9, 
               -np.gradient(np.unwrap(np.angle(meas_compensated['S-parameter'][2])).T)[0][:,21]/ang_freq_step)
#plt.close()
plt.xlabel('Частота, ГГц')
plt.ylabel('Задержка, нс')
plt.tight_layout()
plt.grid(True)
plt.savefig('single-tone-spectroscopy-cs.pdf')

In [38]:
plt.figure(figsize=(12,6))
plt.title('Однотоновая спектроскопия резонатора (0 дБм), образец #{}'.format(qubit_id))
plt.plot(meas_compensated['S-parameter'][1][1]/1e9, 
               -np.gradient(np.unwrap(np.angle(meas_compensated['S-parameter'][2])).T)[0][:,17]/ang_freq_step)
#plt.close()
plt.xlabel('Частота, ГГц')
plt.ylabel('Задержка, нс')
plt.tight_layout()
plt.grid(True)
plt.savefig('single-tone-spectroscopy-cs.pdf')

# Двухтоновая спектроскопия (определение области частот, отвечающих кубитам)

In [71]:
ex_freqs = np.arange(3.0e9, 9.0e9, 5e6)
#lo1.set_power(-10) # no mixer
lo1.set_power(0) # w/mixer
#Измерение
lo1.set_status(1)
pna.set_nop(1)
pna.set_centerfreq(Qubits[qubit_id]['r']['Fr'])
pna.set_frequency(Qubits[qubit_id]['r']['Fr'])
pna.set_bandwidth(20)
pna.set_power(Qubits[qubit_id]['r']['P'])

header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'), 
                       filename = "Двухтоновая спектроскопия образец #{}".format(qubit_id))
except KeyboardInterrupt: pass 
lo1.set_status(0)

A Jupyter Widget

Started at:  Aug 29 2018 16:46:30
Exiting sweep main thread #{} via return 11188

Elapsed time: 0 h 1 m 29.21 s


True

# Двухтоновая спектроскопия при различных мощностях второго (возбуждающего) тона

In [72]:
ex_freqs = np.arange(3.4e9, 4.4e9, 4e6)
#ex_freqs = np.arange(4.08e9, 4.12e9,0.25e6)
ex_powers = np.arange(-20, 14, 1)

pna.set_xlim(Qubits[qubit_id]['r']['Fr'], Qubits[qubit_id]['r']['Fr'])
pna.set_nop(1)
#pna.set_power(Qubits[qubit_id]['r']['P']+10)
#pna.set_power(-30)
pna.set_power(Qubits[qubit_id]['r']['P'])
pna.set_bandwidth(40)
#lo1.set_power(-20)

header = '''#Parameter =
#Ro power = {:f}'''.format( pna.get_power() )

lo1.set_status(1)
#ps.S21vs2DPar(pna, currents, current.set_current, ex_freqs, lo1.set_frequency, filename = "#2{:.1f}dBm".format(lo1.get_power()),\
#           header = header)
ex2D = sweep.sweep(pna, (ex_powers, lo1.set_power, 'Мощность второго тона, дБ'), 
                   (ex_freqs, lo1.set_frequency, 'Частота второго тона, ГГц'), 
                   filename = "Двухтоновая спектроскопия при различных мощностях второго (возбуждающего) тона образец #{}".format(qubit_id),\
           header = header, shuffle=True)
lo1.set_status(0)

A Jupyter Widget

Started at:  Aug 29 2018 16:48:42
Exiting sweep main thread #{} via return 9444

Elapsed time: 0 h 6 m 15.36 s


True

# Точное наблюдение линии 00->11

In [74]:
ex_freqs = np.arange(4.075e9, 4.15e9, 0.1e6)
#ex_freqs = np.arange(4.08e9, 4.12e9,0.25e6)
ex_powers = np.arange(-20, 10, 1)

pna.set_xlim(Qubits[qubit_id]['r']['Fr'], Qubits[qubit_id]['r']['Fr'])
pna.set_nop(1)
#pna.set_power(Qubits[qubit_id]['r']['P']+10)
#pna.set_power(-30)

pna.set_bandwidth(3)
if qubit_id=='2':
    lo1.set_power(5)
    pna.set_power(Qubits[qubit_id]['r']['P']-15)
if qubit_id=='1':
    lo1.set_power(-1.5)
    pna.set_power(Qubits[qubit_id]['r']['P']-15)

header = '''#Parameter =
#Ro power = {:f}'''.format( pna.get_power() )

lo1.set_status(1)
#ps.S21vs2DPar(pna, currents, current.set_current, ex_freqs, lo1.set_frequency, filename = "#2{:.1f}dBm".format(lo1.get_power()),\
#           header = header)
#ex2D = sweep.sweep(pna, (ex_powers, lo1.set_power, 'Мощность второго тона, дБ'), 
#                   (ex_freqs, lo1.set_frequency, 'Частота второго тона, ГГц'), filename = "Двухтоновая спектроскопия (линия 00-11) #{}".format(qubit_id),\
#           header = header, shuffle=True)
sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Частота второго тона, ГГц'), filename = "Двухтоновая спектроскопия (линия 00-11) #{}".format(qubit_id),\
           header = header, shuffle=True)
lo1.set_status(0)

A Jupyter Widget

Started at:  Aug 29 2018 17:01:34
Exiting sweep main thread #{} via return 1564

Elapsed time: 0 h 4 m 4.22 s


True

# Измерить зависимость отражённого от резонатора сигнала от амплитуды считывающего импульса. Определить амплитуду насыщения, выставить амплитуду ниже уровня насыщения.

## Импульсный режим измерений

In [75]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм

lo_ex_pow = 14
lo_ex.set_status(1)
lo_ro_pow = 16
lo_ro.set_power(lo_ro_pow)

pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)

marker_length = 100
readout_trigger_delay = 185
trg_length = 10e-9

ex_clock = 1e9
ro_clock = 1e9

rep_rate = 20e3 # частота повторений эксперимента

awg.stop()
awg.check_cached=True
#awg_tek.set_clock(ex_clock) # клок всех авгшк
awg.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.
#this is zashkvar

awg_tek = awg
# channel 0 is master and triggers all others
# M3202A settings (ignored on AWG5014C)
awg.trigger_source_types = [0, 6,6,6]
awg.trigger_source_channels = [0, 4000,4000,4000] # pxi trigger 0 
awg.trigger_delays = [40, 0,0,0] # master channel should wait 400 ns for others to start
awg.trigger_behaviours = [0,4,4,4] #rising edge trigger
#for channel in range(0,4): #M3202A
for channel in range(1,5): #AWG5014C
    awg.set_amplitude(.2, channel=channel)
    awg.set_offset(0, channel=channel)
    awg.set_output(1, channel=channel)
    awg.set_waveform([0]*awg_tek.get_nop(), channel=channel)

    
#awg.set_marker(length=marker_length, delay=0, channel=0) #M3202A


awg_channels =dict()
ro_trg = awg_digital.awg_digital(awg_tek, 1)
#ro_trg.mode = 'set_delay' #M3202A
#ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-readout_trigger_delay)) #M3202A
ro_trg.mode = 'waveform' #AWG5014C
awg_channels['ro_trg'] = ro_trg
# К первому маркере первого канала тектроникса подсоединён триггер оцифровщика.
# Надо подавать какой-нибудь триггер, что-ли.
# А ещё нужно подавать клок. К сожалению наш оцифровщик не может в 1ГГц клок, ему нужен 500МГц клок.
# Будем подавать 500 МГц с цифрового выхода тектроникса.
# AWG5014C
awg_tek.set_marker1_low(-0.0, channel=1)
awg_tek.set_marker2_low(-0.0, channel=1)
awg_tek.set_marker1_high(1.0, channel=1)
awg_tek.set_marker2_high(1.0, channel=1)
awg_tek.set_marker1_low(-0.04, channel=2)
awg_tek.set_marker2_low(-0.04, channel=2) 
awg_tek.set_marker1_high(0.04, channel=2) # с клока хватит и такой амплитуды
awg_tek.set_marker2_high(0.04, channel=2) # с клока хватит и такой амплитуды

# настройки оцифровщика
# Spectrum M3i-3212
adc.reset()            
adc.set_timeout(10000)
adc.set_clock(ro_clock)

#adc.set_spc_samplerate(smp_rate)
#External trigger
adc.set_trigger_ext0_level0(0) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(1300) # logical 1 value to 600 mV
adc.trigger_termination_50Ohm()
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)
adc.disable_trigger_output()
adc.select_channel01()
adc.set_multi_mode()
# #adc.set_segmentsize()
adc.set_trigger_ORmask_tmask_ext0()
adc.set_trigger_ANDmask_tmask_ext0()
adc.set_trigger_ORmask_tmask_NO_ch0()
adc.set_trigger_ORmask_tmask_NO_ch1()
adc.set_trigger_ANDmask_tmask_NO_ch0()
adc.set_trigger_ANDmask_tmask_NO_ch1()
# # усилитель на оцифровщике
adc.set_input_amp_ch0(50)
adc.set_input_amp_ch1(50)
adc.set_input_offset_ch0(0)
adc.set_input_offset_ch1(0)

# #External clock
# adc.set_reference_clock(adc.get_clock()) #500 MHz clock
adc.set_reference_clock(10e6) #500 MHz clock

#adc.set_input(channel=1, input=1)
#adc.set_input(channel=2, input=1)
#adc.set_trigger_external(channel=1)
#adc.set_trigger_external(channel=2)

True

## Настройки смесителей

In [76]:
ro_if = 75e6
lo_freq = 4.05e9

imp.reload(awg_iq_multi)
# промежуточные частоты для гетеродинной схемы new:
lo1.set_frequency(lo_freq)
ex_if = lo_freq-qubit_params[qubit_id]['q']['00-1-01']
#iq_ex = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 0, 1, lo_ex) #M3202A
iq_ex = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 1, 2, lo_ex) #M3202A
for tr,freq in qubit_params[qubit_id]['q'].items():
    iq_ex.carriers[tr] = awg_iq_multi.carrier(iq_ex)
    iq_ex.carriers[tr].set_frequency(freq)
    awg_channels['iq_ex_'+tr]=iq_ex.carriers[tr]

lo_ro.set_frequency(qubit_params[qubit_id]['r']['Fr']+ro_if)
#iq_ro = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 2, 3, lo_ro) #M3202A
iq_ro = awg_iq_multi.awg_iq_multi(awg_tek, awg_tek, 4, 3, lo_ro) #AWG5014C
iq_ro.carriers['iq_ro'] = awg_iq_multi.carrier(iq_ro)
iq_ro.carriers['iq_ro'].set_frequency(qubit_params[qubit_id]['r']['Fr'])
awg_channels['iq_ro'] = iq_ro.carriers['iq_ro']
readout_channels = {'iq_ro':awg_channels['iq_ro']}

pg = pulses.pulses(awg_channels)
print(ex_if*1e-6 ,lo_freq)

print("Excitation channel", end='\n')
iq_ex.do_calibration(sa)
print("Readout channel", end='\n')
iq_ro.do_calibration(sa)

-200.41199999999998 4050000000.0
Excitation channel
d:\data/calibrations/iq-dc/ lo_freq-4.05e+09
d:\data/calibrations/iq-rf/ frequency-4.25e+09-if-2.004e+08-sideband_id-     0
d:\data/calibrations/iq-rf/ frequency-4.129e+09-if-7.927e+07-sideband_id-     0
d:\data/calibrations/iq-rf/ frequency-4.095e+09-if-4.495e+07-sideband_id-     0
d:\data/calibrations/iq-rf/ frequency-4.015e+09-if--3.53e+07-sideband_id-     0
d:\data/calibrations/iq-rf/ frequency-3.931e+09-if--1.186e+08-sideband_id-     0
d:\data/calibrations/iq-rf/ frequency-3.807e+09-if--2.433e+08-sideband_id-     0
d:\data/calibrations/iq-rf/ frequency-3.676e+09-if--3.738e+08-sideband_id-     0
Readout channel
d:\data/calibrations/iq-dc/ lo_freq-7.407e+09
max() arg is an empty sequence
7407000000.0
[ 0.3  0.3] -57.0158958435
[ 0.315  0.3  ] -56.8946266174
[ 0.3    0.315] -56.8003845215
[ 0.315  0.285] -57.1278038025
[ 0.3225  0.27  ] -57.2783660889
[ 0.3075  0.27  ] -57.4132003784
[ 0.30375  0.255  ] -57.6812973022
[ 0.32625  0.2

dc:  1.01e-01-1.72e-01j	I:  4.00e-01	Q: 3.97e-01-3.80e-02j	B: -104.48 G: -72.54, C:0.00
[ -72.5425415  -103.83494568 -104.48247528]
dc:  1.01e-01-1.72e-01j	I:  4.00e-01	Q: 3.97e-01-3.95e-02j	B: -104.34 G: -72.55, C:0.00
[ -72.54901886 -104.14929962 -104.34441376]
dc:  1.01e-01-1.72e-01j	I:  4.00e-01	Q: 3.97e-01-4.10e-02j	B: -104.39 G: -72.54, C:0.00
[ -72.54249573 -104.42883301 -104.39070129]
dc:  1.01e-01-1.72e-01j	I:  4.00e-01	Q: 3.97e-01-4.28e-02j	B: -104.49 G: -72.53, C:0.00
[ -72.53094482 -102.87387085 -104.49234009]
dc:  1.01e-01-1.72e-01j	I:  4.00e-01	Q: 3.99e-01-4.36e-02j	B: -104.19 G: -72.52, C:0.00
[ -72.52253723 -102.95189667 -104.18798828]
dc:  1.01e-01-1.72e-01j	I:  4.00e-01	Q: 3.97e-01-4.16e-02j	B: -104.41 G: -72.52, C:0.00
[ -72.5209198  -104.12042999 -104.40844727]
dc:  1.01e-01-1.72e-01j	I:  4.00e-01	Q: 3.98e-01-4.29e-02j	B: -104.51 G: -72.53, C:0.00
[ -72.53409576 -103.5861969  -104.50875092]
dc:  1.01e-01-1.72e-01j	I:  4.00e-01	Q: 3.99e-01-4.18e-02j	B: -104.28 G: -72

## задержка считывания

In [77]:
trigger_readout_seq = [pg.p('ro_trg', 10e-9, pg.rect, 1)]

adc.set_nums(50)
adc.set_nop(32768)
if hasattr(adc, 'set_post_trigger'):
    adc.set_post_trigger(adc.get_nop()-64)
modem = modem_readout.modem_readout(pg, adc, trigger_readout_seq, axis_mean=0)
modem.readout_channels = readout_channels
#ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-0)) # M3201A & M3202A
ro_trg.delay = 0#Spectrum+AWG5014C
readout_trigger_delay = modem.calibrate_delay()[awg_channels['iq_ro']]
#readout_trigger_delay_cycles = readout_trigger_delay*adc.get_clock() M3201A
readout_trigger_delay_cycles = readout_trigger_delay*ro_trg.get_clock()
print (readout_trigger_delay, readout_trigger_delay_cycles)
#ro_trg.delay_setter = lambda x: adc.set_trigger_delay(int(x*adc.get_clock()/iq_ex.get_clock()-readout_trigger_delay_cycles)) M3201A
ro_trg.delay = int(-readout_trigger_delay_cycles)#Spectrum+AWG5014C
readout_trigger_delay_test = modem.calibrate_delay()[awg_channels['iq_ro']]
#readout_trigger_delay_cycles_test = readout_trigger_delay_test*adc.get_clock() M3201A
readout_trigger_delay_cycles_test = readout_trigger_delay_test*ro_trg.get_clock()
print (readout_trigger_delay_test, readout_trigger_delay_cycles_test)

modem.calibrate_dc()

-2.02e-07 -202.0
0.0 0.0


{'iq_ro': {'coherent_background': array([-16.00-17.9j , -14.52-16.06j, -13.92-16.8j , ..., -14.54-17.26j,
         -13.60-15.78j, -12.00-16.j  ]),
  'feature': array([ 0.06207503+0.13757878j, -0.09272088+0.18983483j,
         -0.17107496+0.08558544j, ...,  0.15970946-0.13469192j,
          0.14409462+0.03213365j,  0.00968392+0.17246729j]),
  'iq_calibration': array([-0.03630844-0.01106534j,  0.09838347+0.14864412j])}}

## усреднённое считывание

In [78]:
modem.filters = modem.calibrated_filters
adc_reducer = data_reduce.data_reduce(modem)
adc_reducer.filters = {'S21': data_reduce.mean_reducer(modem, 'iq_ro', axis=0)}
adc_reducer.extra_opts['scatter'] = True
adc_reducer.extra_opts['realimag'] = True

## длительность считывания

In [79]:
def set_adc_nop(ro_adc_length):
    adc.stop() # Spectrum
    signal_size = int(np.ceil(4./3.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    #print (nop)
    adc.set_nop(nop)
    adc.set_post_trigger(adc.get_nop()-64)
    
# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
    
    
#ro_adc_length = 0.15e-6        # Вот такой длины считываем (опухоль Фёдорова!!)
#ro_amplitude =0.16
#ro_dac_length = 0.08e-6
ro_adc_length = 0.1e-6
ro_dac_length = 0.08e-6
#adc.set_nums(1000)
#adc.set_software_nums_multi(1)
adc.set_nums(100)
set_adc_nop(ro_adc_length)
adc.set_nums(10000)

True

## измерение среднего прошедшего импульса

In [80]:
mean_sample = data_reduce.data_reduce(adc)
mean_sample.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)

#adc_reducer_old = data_reduce.data_reduce(adc)
#adc_reducer_old.filters['Std Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage std', 0)
mean_sample.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.carriers['iq_ro'].get_if())
mean_sample.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.carriers['iq_ro'].get_if())
#adc_downsampler = data_reduce.data_reduce(adc)
#adc_downsampler.filters['Voltage'] = data_reduce.downsample_reducer(adc, 'Voltage', 0, iq_ro.get_if(), 4)
# # Этот измеритель мы как правило используем когда точек не слишком много и все результаты его жизнедеятельности как правило 
# # выглядят как ломаные. Чтобы было красиво, давайте лучше сделаем точки (а кривые потом получим фитованые)
mean_sample.extra_opts['scatter'] = True

awg.check_cached = True

## Прохождение считывающих импульсов разной амплитуды

In [81]:
#%%mprun -f sweep.sweep
imp.reload(sweep)
imp.reload(plotting)
mean_sample.filters['Std Voltage (AC)'] = data_reduce.std_reducer_noavg(adc, 'Voltage', 0, 1)
def set_pulse_amplitude(x):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    pg.set_seq(trigger_readout_seq+[pg.p('iq_ro', ro_dac_length, pg.rect, x)])
    awg_tek.run()
measurement = sweep.sweep(mean_sample, (np.linspace(0, 1.0, 21), set_pulse_amplitude, 'Амплитуда считывания'), filename='Прохождение считывающих импульсов образец #{}'.format(qubit_id))
del mean_sample.filters['Std Voltage (AC)']

A Jupyter Widget

Started at:  Aug 29 2018 17:12:56
Waiting to join deferred threads:
Exiting sweep main thread #{} via return 332

Elapsed time: 0 h 0 m 53.86 s


In [82]:
zero_response = measurement['Mean Voltage (AC)'][2][0,:]
drive_amplitudes = measurement['Mean Voltage (AC)'][1][0][1:]
signal = measurement['Mean Voltage (AC)'][2][1:,:]
noise = measurement['Std Voltage (AC)'][2][1:,:]
error = noise/np.sqrt(adc.get_nums())
signal_overlap = np.sum(np.conj(signal[0,:])*signal[1:,:], axis=1)/drive_amplitudes[1:]
signal_overlap_estimate = np.real(signal_overlap[0])
signal_overlap_error = 0.5*np.sqrt(np.sum((np.abs(signal[1:,:])*error[0,:])**2, axis=1)+np.sum((np.abs(signal[0,:])*error[1:,:])**2,axis=1))/drive_amplitudes[1:]
#signal_overlap_estimate = (np.sum(np.abs(signal[0,:])**2) - np.sum(error[0,:]*np.abs(signal[0,:]))-np.sum(np.abs(error[0,:])**2))/drive_amplitudes[0]
#plt.figure()
#plt.plot(np.real(signal_overlap))
#plt.plot(np.sum(noise**2, axis=1)/adc.get_nums())
#plt.plot(signal_overlap_error)
compression = 10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate))
db_compression_point = np.argmax(10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate))<-1+10*np.log10(1-signal_overlap_error/signal_overlap_estimate))
#10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate)),-1+10*np.log10(1-signal_overlap_error/signal_overlap_estimate)
#10*np.log10(np.real(signal_overlap)/np.real(signal_overlap_estimate))<-1+10*np.log10(1-signal_overlap_error/signal_overlap_estimate)
if np.any(db_compression_point):
    ro_amplitude = drive_amplitudes[db_compression_point+1]
else:
    ro_amplitude = drive_amplitudes[-1]
print("Readout amplitude:",ro_amplitude)    

Readout amplitude: 1.0


In [83]:
plt.figure('Компрессия считывающего импульса')
plt.plot(drive_amplitudes[1:], compression)
plt.xlabel('Амплитуда считывания')
plt.ylabel('Компрессия, дБ')

Text(38.2222,0.5,'Компрессия, дБ')

In [ ]:
ro_parameters = {"adc_len":ro_adc_length,'dac_len': ro_dac_length,'amp': ro_amplitude,'adc nums':adc.nums,
                'trg_length':trg_length}
qjson.dump("setups","readout",ro_parameters)

In [56]:
ro_amplitude = 1.0
ro_sequence = trigger_readout_seq+[pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]

## Раби+Рамзи

In [84]:
import qsweepy.qjson as pqjson
import qsweepy.quantum_two_level_dynamics as dyn

In [85]:
imp.reload(dyn)
imp.reload(qjson)
imp.reload(data_reduce)
imp.reload(plotting)
imp.reload(fitting)
imp.reload(sweep)
imp.reload(diff_readout)

<module 'qsweepy.diff_readout' from 'D:\\qtlab_replacement\\qsweepy\\diff_readout.py'>

In [86]:
#ex_channels = ['iq_ex_00-1-01', 'iq_ex_00-1-10', 'iq_ex_01-1-02']#, 'iq_ex_00-2-02']
ex_channels = ['iq_ex_00-1-01', 
               'iq_ex_00-2-02', 
               #'iq_ex_00-2-11', 
               'iq_ex_00-1-10', 
               'iq_ex_00-2-20', 
              ]

ex_amplitudes = {'iq_ex_00-1-01':1, 
               'iq_ex_00-2-02':1, 
               #'iq_ex_00-2-11':1, 
               'iq_ex_00-1-10':1, 
               'iq_ex_00-2-20':1, 
                }
rabi_lengths = {'iq_ex_00-1-01':np.arange(0., 100e-9, 2e-9), 
                'iq_ex_00-2-02':np.arange(0., 500e-9, 10e-9), 
                #'iq_ex_00-2-11':np.arange(0., 10000e-9, 100e-9), 
                'iq_ex_00-1-10':np.arange(0., 100e-9, 2e-9), 
                'iq_ex_00-2-20':np.arange(0., 500e-9, 10e-9),
                }
delaysT2_coherence = {'iq_ex_00-1-01':np.arange(0., 5e-6, 50e-9), 
                      'iq_ex_00-2-02':np.arange(0., 1e-6, 20e-9), 
                      #'iq_ex_00-2-11':np.arange(0., 10e-6, 100e-9), 
                      'iq_ex_00-1-10':np.arange(0., 5e-6, 50e-9), 
                      'iq_ex_00-2-20':np.arange(0., 1e-6, 20e-9),
                     }
ro_par = {'qubit_id':qubit_id,'ex_channels':ex_channels}
fitter = lambda x,y: fitting.fit1d(x,y,'S21')
ro_par['fitter']=fitter

#rabi_lengths = np.arange(0., 2000e-9, 5e-9)
delaysT2 = np.arange(0, 200e-9, 2e-9)
target_freq_offset = -20e6
#delaysT2_coherence = np.arange(0, 5000e-9, 10e-9)
target_freq_offset_coherence = -2e6
delaysT1 = np.arange(0, 15e-6,0.05e-6)

In [87]:
imp.reload(plotting)

<module 'qsweepy.plotting' from 'D:\\qtlab_replacement\\qsweepy\\plotting.py'>

In [88]:
imp.reload(diff_readout)
diff_adc_reducer = diff_readout.diff_readout(adc_reducer)

In [89]:
twos = {ex_channel:dyn.quantum_two_level_dynamics(pg,
                                           adc_reducer,
                                           ex_channel=ex_channel,
                                           ro_channel='iq_ro',
                                           ro_sequence=ro_sequence,
                                           ex_amplitude=ex_amplitudes[ex_channel],
                                           fitter=fitter, 
                                           shuffle=True,
                                           qubit_id=qubit_id) for ex_channel in ex_channels}

d:\data/calibrations/setups/ readout
Failed loading rabi frequency calibration: 'quantum_two_level_dynamics' object has no attribute 'ex_amplitude'
d:\data/calibrations/setups/ readout
Failed loading rabi frequency calibration: 'quantum_two_level_dynamics' object has no attribute 'ex_amplitude'
d:\data/calibrations/setups/ readout
Failed loading rabi frequency calibration: 'quantum_two_level_dynamics' object has no attribute 'ex_amplitude'
d:\data/calibrations/setups/ readout
Failed loading rabi frequency calibration: 'quantum_two_level_dynamics' object has no attribute 'ex_amplitude'


In [91]:
#%%mprun -f two_diffs[0].Rabi_rect -f two_diffs[0].Ramsey -f two_diffs[0].decay -f two_diffs[0].spin_echo -f sweep.sweep -f diff_adc_reducer.measure
two_diffs = twos
imp.reload(plotting)
iters = 1

for _iter in range(iters):
    for ex_channel, two_diff in two_diffs.items():
        two_diff.Rabi_rect(rabi_lengths[ex_channel])
        #two_diff.Ramsey(delaysT2, target_freq_offset)
        two_diff.Ramsey(delaysT2_coherence[ex_channel],target_freq_offset_coherence)
        #two_diff.decay(delaysT1)
        #two_diff.spin_echo(delaysT2_coherence,target_freq_offset_coherence)

A Jupyter Widget

Started at:  Aug 29 2018 17:15:42
Waiting to join deferred threads:
Exiting sweep main thread #{} via return 10104

Elapsed time: 0 h 2 m 7.2 s


A Jupyter Widget

Started at:  Aug 29 2018 17:17:53
Waiting to join deferred threads:
Exiting sweep main thread #{} via return 9540

Elapsed time: 0 h 4 m 17.94 s


A Jupyter Widget

...

Started at:  Aug 29 2018 17:31:49
Waiting to join deferred threads:
Exiting sweep main thread #{} via return 3972

Elapsed time: 0 h 6 m 22.47 s


A Jupyter Widget

Started at:  Aug 29 2018 17:38:15
Waiting to join deferred threads:
Exiting sweep main thread #{} via return 1784

Elapsed time: 0 h 3 m 8.19 s


A Jupyter Widget

Started at:  Aug 29 2018 17:41:27
Waiting to join deferred threads:
Exiting sweep main thread #{} via return 2232

Elapsed time: 0 h 3 m 7.55 s
